In [1]:
import csv
import pandas as pd
from wikimedia import searchEntity, searchObjWProperty, searchProperty
from dateutil.parser import parse
import operator
from indexer import connect_elasticsearch, search
import logging
import json

In [382]:
def lDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [383]:
property_index = 'wd_property'
entity_index = 'wd_entity'
property_doc_type = 'members'

In [821]:
df = pd.read_csv("data/dataKicut/ruang-laktasi.csv", encoding='latin-1')
protagonist = 'Nama_fasilitas'
df

,nama_rptra,kecamatan,kelurahan,alamat,kota,tanggal_peresmian,ketersediaan_ruang _laktasi
0,RPTRA Sungai Bambu,Tanjung Priok,Sungai Bambu,Jl. Jati Raya Rw 06,Jakarta Utara,13 Mei 2015,Ya
1,RPTRA Bahari,Cilandak,Gandaria Selatan,Jl. Bahari Raya Rt 009 Rw 07,Jakarta Selatan,21 Mei 2015,Ya
2,RPTRA Kenanga,Gambir,Cideng,Jl. Makian No. 1 Rt 002/ 005,Jakarta Pusat,30 Mei 2015,Ya
3,RPTRA Kembangan,Kembangan,Kembangan Utara,Jl. Gang Kompas Rt 007,Jakarta Barat,5 Juni 2015,Ya
4,RPTRA Amiterdam,Kepulauan Seribu Selatan,Untung Jawa,Pulau Untung Jawa,Kepulauan Seribu,10 Oktober 2015,Tidak
5,RPTRA Cililitan,Kramat Jati,Cililitan,Jl. Buluh Rt 10 Rw 16,Jakarta Timur,22 Oktober 2015,Ya
6,RPTRA Sunter Jaya Berseri,Tanjung Priok,Sunter Jaya,"RT 005, 006, 013 RW 05",Jakarta Utara,18 Desember 2015,Ya
7,RPTRA Pulogebang Indah,Cakung,Pulo Gebang,Rusun Pulo Gebang,Jakarta Timur,23 Desember 2015,Ya
8,RPTRA Meruya Utara,Kembangan,Meruya Utara,SMPN 215 RT 017/04,Jakarta Barat,29 Desember 2015,Ya
9,RPTRA Intiland Teduh,Tanah Abang,Karet Tengsin,Jl Karet Pasar Baru Barat I,Jakarta Pusat,30 Desember 2015,Ya


In [822]:
header_list = list(df)
header_list


['nama_rptra',
 'kecamatan',
 'kelurahan',
 'alamat',
 'kota',
 'tanggal_peresmian',
 'ketersediaan_ruang _laktasi']

In [823]:
logging.basicConfig(level=logging.ERROR)
es = connect_elasticsearch()

Connected to ES Service


In [824]:
map_list = []
id_list = []
score_list = []
for elem in header_list:
    elem = elem.replace('_', ' ')
    search_object = {
    'query': {
        'multi_match': {
            'query': elem,
            'fields': 'alias'
        }
    }
}
    
    res = search(es, property_index, json.dumps(search_object))['hits']['hits']
    if(len(res) > 0):
        found = False
        for cand in res:
            if(elem.lower() in cand['_source']['label']):
                map_list.append(cand['_source']['label'])
                id_list.append(cand['_source']['id'])
                found = True
                break
        if(not found):
            for cand in res:
                if(elem.lower() in cand['_source']['alias']):
                    map_list.append(cand['_source']['label'])
                    id_list.append(cand['_source']['id'])
                    found = True
                    break
        if(not found):
            split = elem.split()
            for word in split:
                for cand in res:
                    for candword in cand['_source']['alias']:
                        if(word.lower() in candword):
                            map_list.append(cand['_source']['label'])
                            id_list.append(cand['_source']['id'])
                            found = True
                            break
                    if(found):
                        break
                if(found):
                    break
        if(not found):
            for cand in res:
                if(elem.lower() in cand['_source']['description']):
                    map_list.append(cand['_source']['label'])
                    id_list.append(cand['_source']['id'])
                    found = True
                    break
        if(not found):
            minDist = 10000
            candItem = None
            temp = 0
            for cand in res:
                for alt in cand['_source']['alias']:
                    temp = min(minDist, lDistance(elem.lower(), alt))
                if(temp < minDist):
                    candItem = cand
                    minDist = temp
            
            if(candItem['_score'] > 18 and minDist < 8):
                map_list.append(candItem['_source']['label'])
                id_list.append(candItem['_source']['id'])
                score_list.append(candItem['_score'])
            else:
                map_list.append('')
                id_list.append('')
                score_list.append(candItem['_score'])
                
    else:
        map_list.append('')
        id_list.append('')
        score_list.append('')

In [825]:
header_list

['nama_rptra',
 'kecamatan',
 'kelurahan',
 'alamat',
 'kota',
 'tanggal_peresmian',
 'ketersediaan_ruang _laktasi']

In [826]:
map_list

['nama pengguna situs web',
 'terletak di wilayah administrasi',
 'terletak di wilayah administrasi',
 'alamat',
 'ibu kota',
 'sejak',
 '']

In [734]:
id_list

['', 'P969', '', '', 'P131', 'P131', 'P281', 'P2588', '', 'P1329']

In [336]:
score_list

[23.67025, 16.668255, 25.746075, 29.885]

In [656]:
key = 'wilayah'
search_object = {
  "query": { 
    "bool": { 
      "should": [
        { 
            "match": { 
                "alias":  {
                    "query": key
                } 
            }
        }
      ]
    }
  }
}

res = search(es, property_index, json.dumps(search_object))['hits']['hits']
res

[{'_index': 'wd_property',
  '_type': 'members',
  '_id': 'fkblWGkBzF5h07iCZjmO',
  '_score': 49.685196,
  '_source': {'label': 'kode wilayah Kemendagri',
   'id': 'P2588',
   'alias': ['kode wilayah',
    'kode desa Indonesia',
    'kode desa Kemendagri',
    'kode kabupaten',
    'kode kecamatan',
    'kode Kemendagri',
    'kode provinsi',
    'kode wilayah desa Indonesia',
    'kode wilayah Indonesia',
    'kode wilayah Kemendagri'],
   'description': 'kode penanda unik untuk pembagian wilayah administrasi setingkat desa, kecamatan, kabupaten, dan provinsi di Republik Indonesia oleh Kementrian Dalam Negeri tahun 2015',
   'data type': 'ExternalId'}},
 {'_index': 'wd_property',
  '_type': 'members',
  '_id': '1EbkWGkBzF5h07iC9jP3',
  '_score': 44.054485,
  '_source': {'label': 'wilayah layanan bandara',
   'id': 'P931',
   'alias': ['wilayah layanan bandara', 'wilayah layanan bandara'],
   'description': 'kota atau wilayah yang dilayani negara terkait',
   'data type': 'WikibaseItem

In [208]:
map_list = []
id_list = []
for elem in header_list:
    elem = elem.replace('_', ' ')
    search_object = {
        'query': {
            'multi_match': {
                'query': elem,
                'fields': ['alias', 'label', 'description'],
                'fuzziness': 'AUTO'
            }
        }
    }
    
    res = search(es, property_index, json.dumps(search_object))['hits']['hits']
    if(len(res) > 0):
        map_list.append(res[0]['_source']['label'])
        id_list.append(res[0]['_source']['id'])
    else:
        map_list.append('')
        id_list.append('')

In [708]:
search_object = {
    "from" : 0, "size" : 20,
    'query': {
        'multi_match': {
            'query': 'kota_administrasi',
            'fields': 'alias'
        }
    }
} 
res = search(es, property_index, json.dumps(search_object))['hits']['hits']
res

[{'_index': 'wd_property',
  '_type': 'members',
  '_id': 'akbkWGkBzF5h07iC2DHC',
  '_score': 125.23839,
  '_source': {'label': 'terletak di wilayah administrasi',
   'id': 'P131',
   'alias': ['desa',
    'jorong',
    'kabupaten',
    'kampung',
    'kecamatan',
    'kelurahan',
    'kotamadya',
    'lokasi administrasi',
    'nagari',
    'terletak di entitas teritorial administratif',
    'terletak di tingkat administrasi',
    'tingkat administrasi',
    'wilayah administrasi',
    'provinsi',
    'terletak di wilayah administrasi'],
   'description': 'butir tersebut berada di wilayah entitas administratif berikut. Gunakan P276 (lokasi) untuk menentukan lokasi tempat non-administratif dan untuk butir tentang peristiwa',
   'data type': 'WikibaseItem'}},
 {'_index': 'wd_property',
  '_type': 'members',
  '_id': 'dUbkWGkBzF5h07iC2TEx',
  '_score': 51.07329,
  '_source': {'label': 'memiliki entitas teritorial administratif',
   'id': 'P150',
   'alias': ['terdiri dari',
    'terbagi 

In [636]:
search_object = {
    'query': {
        'multi_match': {
            'query': 'luas tanah',
            'fields': ['alias', 'label'],
            'fuzziness': 'AUTO'
        }
    }
}

res = search(es, property_index, json.dumps(search_object))['hits']['hits']
print(res)
if(len(res) > 0):
    found = False
    for cand in res:
        if(elem.lower() in cand['_source']['label']):
            print(cand['_source']['label'])
            print(cand['_source']['id'])
            found = True
            break
    if(not found):
        for cand in res:
            if(elem.lower() in cand['_source']['alias']):
                print(cand['_source']['label'])
                print(cand['_source']['id'])
                found = True
                break
    if(not found):
        for cand in res:
            if(elem.lower() in cand['_source']['description']):
                print(cand['_source']['label'])
                print(cand['_source']['id'])
                found = True
                break
    if(not found):
        minDist = 10000
        candItem = None
        temp = 0
        for cand in res:
            for alt in cand['_source']['alias']:
                temp = min(minDist, lDistance(elem.lower(), alt))
            temp = min(temp, lDistance(elem.lower(), cand['_source']['label']))
            if(temp < minDist):
                candItem = cand
                minDist = temp

        if(candItem['_score'] > 18):
            print(cand['_source']['label'])
            print(cand['_source']['id'])
            print(candItem['_score'])
        else:
            print('')
            print('')
            print('')

else:
    print('')
    print('')
    print('')

[{'_index': 'wd_property', '_type': 'members', '_id': 'LEbGWGkBzF5h07iC7BpS', '_score': 33.279964, '_source': {'label': 'terletak di wilayah administrasi', 'id': 'P131', 'alias': ['daerah', 'desa', 'jorong', 'kabupaten', 'kampung', 'kecamatan', 'kelurahan', 'kotamadya', 'lokasi administrasi', 'nagari', 'terletak di entitas teritorial administratif', 'terletak di tingkat administrasi', 'tingkat administrasi', 'wilayah administrasi'], 'description': 'butir tersebut berada di wilayah entitas administratif berikut. Gunakan P276 (lokasi) untuk menentukan lokasi tempat non-administratif dan untuk butir tentang peristiwa', 'data type': 'WikibaseItem'}}, {'_index': 'wd_property', '_type': 'members', '_id': 'XUbGWGkBzF5h07iC7hpv', '_score': 31.267756, '_source': {'label': 'berbatasan dengan wilayah perairan', 'id': 'P206', 'alias': ['berbatasan dengan sungai', 'berbatasan dengan teluk', 'disamping danau', 'disamping teluk', 'disamping tepi pantai', 'berbatasan dengan danau', 'berbatasan dengan 